**PENCARIAN DAN PENAMBANGAN WEB - TUGAS 2 : Membuat VSM dan Menghitung TF-IDF berdasarkan hasil crawling berita**

Nama : Isnita Widyur Rahmah
NIM : 220411100048
Kelas : IF 7A

Link Project : https://github.com/taawdyy/ppw

**Crawling Berita Online (CNN Indonesia)**

Crawling berita adalah proses otomatisasi pengambilan data dari situs web berita atau sumber informasi lainnya untuk tujuan analisis, penyimpanan, atau penyajian informasi. Proses ini dilakukan oleh program atau skrip yang dikenal sebagai web crawler atau spider

Import Library

In [1]:
pip install requests beautifulsoup4 pandas


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

requests digunakan untuk mengambil halaman web, BeautifulSoup untuk mengekstrak informasi dari konten HTML, dan pandas untuk menyimpan dan mengolah data yang diambil.

Mendefinisikan URL

In [3]:
category_urls = [
    'https://www.cnnindonesia.com/ekonomi/indeks/5',
    'https://www.cnnindonesia.com/olahraga/indeks/7'
]

Mengumpulkan Data

In [4]:
def crawl_article(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Mengambil judul
    title_element = soup.find('h1', class_='text-cnn_black')
    title = title_element.get_text().strip() if title_element else 'Judul tidak ditemukan'

    # Mengambil Isi
    content_div = soup.find('div', class_='detail-text')
    if content_div:
        paragraphs = content_div.find_all('p')
        content = "\n".join([p.get_text().strip() for p in paragraphs])
    else:
        content = 'Isi artikel tidak ditemukan'

    # Mengambil tanggal
    date_div = soup.find('div', class_='text-cnn_grey text-sm mb-4')
    date_text = date_div.text.strip() if date_div else 'Tanggal tidak ditemukan'

    # Mengambil kategori
    cattegory_div = soup.find('a', class_="text-sm text-cnn_black_light3 hover:text-cnn_red gtm_breadcrumb_kanal")
    cattegory = cattegory_div.text.strip() if cattegory_div else 'Kategori tidak ditemukan'

    return {'Judul': title, 'Isi': content, 'Tanggal': date_text, 'Kategori': cattegory}

Melakukan Crawling Web

In [5]:
def crawl_page(url, article_count, max_articles, articles):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    article_links = soup.find_all('a', class_='flex group items-center gap-4')
    for link in article_links:
        if article_count >= max_articles:
            return article_count
        article_url = link['href']
        article = crawl_article(article_url)
        articles.append(article)
        article_count += 1

    next_button = soup.find('a', class_='inline-flex items-center justify-center w-[30px] h-[30px]')
    next_url = next_button['href'] if next_button else None

    if next_url and article_count < max_articles:
        return crawl_page(next_url, article_count, max_articles, articles)

    return article_count


Mengumpulkan Artikel Berita dari berbagai Kategori

In [6]:
def crawl_category(url, max_articles):
    articles = []
    article_count = 0
    print(f'\nMengambil data dari {url}')
    article_count = crawl_page(url, article_count, max_articles, articles)
    return articles

Mengambil Artikel Berita dari beberapa Kategori yang telah ditentukan dalam category_urls.

In [7]:
max_articles_per_category = 50
all_articles = []

for url in category_urls:
    articles = crawl_category(url, max_articles_per_category)
    all_articles.extend(articles)


Mengambil data dari https://www.cnnindonesia.com/ekonomi/indeks/5



Mengambil data dari https://www.cnnindonesia.com/olahraga/indeks/7


Konversi ke dalam DataFrame

In [8]:
df = pd.DataFrame(all_articles)
df

,Judul,Isi,Tanggal,Kategori
0,Karyawan Boeing Akhiri Mogok Kerja usai Sepaka...,Ribuan pekerja Boeing akhirnya mengakhiri mogo...,"Selasa, 05 Nov 2024 22:06 WIB",Kategori tidak ditemukan
1,"Pakar: BPA Bahaya Bagi Kesehatan, Jangan Diang...",Bahaya paparan Bisphenol A (BPA) dalam wadah p...,"Selasa, 05 Nov 2024 21:57 WIB",Kategori tidak ditemukan
2,Respons AHY dan Nusron soal Pelantikan Basuki ...,Menteri Koordinator Infrastruktur dan Pembangu...,"Selasa, 05 Nov 2024 21:28 WIB",Kategori tidak ditemukan
3,Airlangga soal Ekonomi Kuartal III Tumbuh di B...,Menteri Koordinator Bidang Perekonomian Airlan...,"Selasa, 05 Nov 2024 21:03 WIB",Kategori tidak ditemukan
4,"Tak Lagi BUMN, Bulog Bakal Jadi Badan Otonom d...",Direktur Utama Perum Bulog Wahyu Suparyono men...,"Selasa, 05 Nov 2024 20:46 WIB",Kategori tidak ditemukan
...,...,...,...,...
95,Malaysia Gagal Lolos Semifinal Piala AFF Futsa...,Thailand berhasil mengalahkan Malaysia 3-1 dal...,"Selasa, 05 Nov 2024 19:58 WIB",Kategori tidak ditemukan
96,Akmal: Indonesia Perlu Anut Paspor Ganda untuk...,"Pengamat sepak bola nasional, Akmal Marhali, m...","Selasa, 05 Nov 2024 19:33 WIB",Kategori tidak ditemukan
97,Prediksi Susunan Pemain Real Madrid vs AC Mila...,Real Madrid akan menjamu AC Milan dalam lanjut...,"Selasa, 05 Nov 2024 19:10 WIB",Kategori tidak ditemukan
98,"Resmi, Final Martin vs Bagnaia di MotoGP Catal...",Dorna Sports selaku penyelenggara MotoGP menet...,"Selasa, 05 Nov 2024 18:47 WIB",Kategori tidak ditemukan


Simpan dalam bentuk CSV

In [9]:
df.to_csv('crawl_berita.csv', index=False, encoding='utf-8')